### DEV_FUNCTIONS

In [3]:
### Functions process at local m

def compare_header_in_files(col_tmp,col):
    
    print('***** { Function } : compare_header_in_files() ')
    
    print('')
    
    from collections import Counter
    
    state=Counter(col_tmp)==Counter(col)
    
    if state==True:
        
        print('---> True ')

    else : 
        
        print('---> False ')
  
    return state


def compare_header_dataframe_to_table(l1,l2):
    
    print('***** { Function } : compare_header_dataframe_to_table() ')
    
    print('')
    
    from collections import Counter
    
    state=Counter(l1)==Counter(l2)
    
    if state==True:
        
        print('---> True ')
        
        
    else : 
        
        print('---> False ')
        
    return state


def unidecode_header(df):
    
    print('***** { Function } : unidecode_header() ')
    
    print('')

    from unidecode import unidecode

    df.rename(columns={col: unidecode(col) for col in df.columns}, inplace=True)
    
    return df

     
def prepare_header_to_mysql(df):
    
    print('***** { Function } : prepare_header_to_mysql() ')
    
    print('')
    df=unidecode_header(df)
    
    import re

    new_header = [re.sub('[^0-9a-zA-Z_]+', '_', col) for col in df.columns]

    new_header = [col[:64] for col in new_header]

    new_header = [col if col[0].isalpha() else "col_" + col for col in new_header]

    df.columns = new_header
    
    df.columns=[x.upper() for x in df.columns]

    df.columns=[x[:-1] if x.endswith("_") else x for x in df.columns]
    
    return df


def store_data_to_local(db,df,label):
    
    print('***** { Function } : store_data_to_local() ')
    
    print('')    
            
    Date=df['DATE'].min()

    Date=Date.replace('-','')
    
    Date=Date[:8]
    
    db=db.replace('db','_').replace('_raw','/')
        
    output='data/output_' +db+ label +'_'+ Date+ '.xlsx'
        
    print('---> local : ',output)
    
    print('')  
    
    head=df.head()
    
    print('dataframe-head : \n', head)
    
    print('') 
    
    tail=df.tail()
    
    print('dataframe-tail : \n', tail)
    
    print('') 
       
    df.to_excel(output,index=False)
    
    return Date


def warning(date1,date2):

    print('*'*50)
    
    print('')
    
    print (' ===> Warning : file changes header-columns name !')
    
    print('')
    
    print (' Date : ',date1+' - ',date2)
    
    print('')
          
    print('*'*50)


def conclude(n):
    
    print(' _ '*30)
    
    print('')
    
    print('===> total :  ', str(n) + ' dataframes.')
    
    print('')
    
    print('________________________________________________________________________________________________________________________________________________________________________')
    
    print('')


def get_table_name_latest_in_record(db,query):
    
    print('***** { Function } :  get_table_name_latest_in_record(query): ')
    
    print('')    
    
    import pymysql

    # Establish a connection to the MySQL database
    connection = pymysql.connect(host='localhost',
                                user='t',
                                password='a',
                                db=db)

    # Create a cursor object

    cursor = connection.cursor()

    # Execute a query

    #query='SELECT table_name FROM record WHERE id = (SELECT max(id) FROM record WHERE label = "' + label + '" )'
    
    print('---> execute query : ', query)
    
    print('')

    # value = 'your_value'

    cursor.execute(query)

    # Fetch the results

    results = cursor.fetchone()

    # Print the results
    
    table_name=results[0]

    print('---> table name latest : ', table_name)
    
    print('')

    # Close the cursor and the database connection

    cursor.close()

    connection.close()
    
    return table_name
    

def get_header_from_table_in_mysql(db,query):
    
    print('***** { Function } :  get_header_from_table_in_mysql() ')
    
    print('')

    import pymysql

    # Establish a connection to the MySQL database
    
    connection = pymysql.connect(host='localhost',
                                user='t',
                                password='a',
                                db=db)

    # Create a cursor object

    cursor = connection.cursor()

    # Execute a query

    # query = "SHOW COLUMNS FROM table_name"
    
    print('---> execute query : ', query)
    
    print('')

    # value = 'your_value'

    cursor.execute(query)

    # Fetch the results

    results = cursor.fetchall()

    l1=[]

    # Print the results

    for row in results:
        
        l1.append(row[0])

    print('---> header of table in mysql : ',l1)
    
    print('')

    # Close the cursor and the database connection

    cursor.close()

    connection.close()
    
    return l1


def store_data_to_mysql(db,df,table_name):

    print('***** { Function } : store_data_to_mysql() ')
    
    print('')  
    
    from sqlalchemy import create_engine
    
    # Create SQLAlchemy engine to connect to MySQL Database

    hostname="localhost"
    dbname=db     
    uname="t"                                   
    pwd="a"

    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                        .format(host=hostname, db=dbname, user=uname, pw=pwd))

    # store dataframe to mysql 
    
    df.to_sql(table_name, engine, index=False,if_exists='append')
    
    print('---> stored data to mysql successful, table name : ', table_name )
    
    print('')  


def update_table_name_to_record(db,table_name_new,label):
    
    print('***** { Function } : update_table_name_to_record() ')
    
    print('')  
    
    import pandas as pd
    
    import pymysql

    # Establish a connection to the MySQL database
    connection = pymysql.connect(host='localhost',
                                user='t',
                                password='a',
                                db=db)

    # Create a cursor object

    cursor = connection.cursor()

    # Execute a query

    import datetime
 
    current_time = datetime.datetime.now()
    
    current_time=str(current_time)
    
    current_time=current_time[:16]

    query="INSERT INTO record(table_name, label , date_time) VALUES "+ "( '" + table_name_new + "' , '" + label + "' , '" + current_time + "' );"
    
    print('---> execute query : ', query)
    
    print('')
    # value = 'your_value'

    cursor.execute(query)

    connection.commit()

    # Define the SQL query to read data from the table
    query = "SELECT * FROM record"
    
    print('---> execute query : ', query)
    
    print('')

    # Execute the read query and fetch the results
    cursor.execute(query)
    
    results = cursor.fetchall()

    # Create a Pandas dataframe from the query results
    df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])

    print('---> record : ')
    
    print('')

    # Close the database connection and return the dataframe
    connection.close()
    
    return print(df)
    

def update_to_mysql(db,df,label,Date):
    
    print('***** { Function } : update_to_mysql() ')
    
    print('')

    query='SELECT table_name FROM record WHERE id = (SELECT max(id) FROM record WHERE label = "' + label + '" )'

    table_name=get_table_name_latest_in_record(db,query)
    
    print('')

    query='SHOW COLUMNS FROM ' + table_name  

    l1=get_header_from_table_in_mysql(db,query)
    
    print('')
           
    l2=[col for col in df.columns] 
    
    print('---> header of table in external : ',l2)
    
    print('')

    status=compare_header_dataframe_to_table(l1,l2)
    
    print('')

    if status == True:        
            
        store_data_to_mysql(db,df,table_name)
        
        print('')
        
    else:
           
        table_name_new=label+'_'+Date
                        
        update_table_name_to_record(db,table_name_new,label)
        
        print('') 

        store_data_to_mysql(db,df,table_name_new)
        
        print('')


def covert_date_str(date):
    
    date=date.replace('/','-').replace('_','-')
        
    tmp=date.split('-')
    
    if len(tmp[0])==4:
        
        month=tmp[1]
            
        datex=tmp[-1]
            
        if len(month)==1:
            
            month='0'+month
                
        if len(datex)==1:
            
            datex='0'+datex
                
        date=tmp[0]+'-'+month+'-'+datex
        
    if len(tmp[-1])>=4:
        
        month=tmp[1]
            
        datex=tmp[0]
            
        if len(month)==1:
            
            month='0'+month
                
        if len(datex)==1:
            
            datex='0'+datex
                
        date=tmp[-1][:4]+'-'+month+'-'+datex
  
        
    return date


### DEV_KEYMETRICS

In [9]:
import pandas as pd

import glob

from collections import Counter

from dev_functions import*

import warnings
warnings.filterwarnings('ignore')

db='key_metrics_raw'

def laz_biz(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        a=pd.read_excel(i,header=0)
        
        date=a.columns[0].split(':')[-1].split('_')[-1]
        
        df=pd.read_excel(i, header=5)
        
        col=[col for col in df.columns]
        
        df['Date']=[date]*df.shape[0]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date,date)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')


def shp_biz(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')
    
    col_tmp=[]

    dataframe={}

    n=0
    
    print('### 1. Process files ')
    
    print('')
    
    for i in p:
        
        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i)
        
        col=[col for col in df.columns]
        
        date=i.split('/')[-1].split('_')[-1].split('.')[0]
        
        df['Date']=[date]*df.shape[0]
        
        df['Date']=[x[:4]+'-'+x[4:6]+'-'+x[6:] for x in df['Date'].tolist()]
        
        state=compare_header_in_files(col,col_tmp)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
            
        else : 
            
            n+=1
            
            dataframe[n]=df
            
            warning(date,date)
            
            print('')
            
            col_tmp=col

    conclude(5)
    
    print('')
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
        
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')
        
        #update_to_mysql(db,df,label,Date)
        
        print('')


def shp_overall(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')
    
    col_tmp=[]

    dataframe={}

    n=0
    
    print('### 1. Process files ')
    
    print('')
    
    for i in p:
        
        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i,sheet_name='Placed Order',dtype=str, skiprows=3)
        
        df['Date']=[x.split('-')[-1]+'-'+x.split('-')[1]+'-'+x.split('-')[0] for x in df['Date'].tolist()]
        
        date1=df['Date'].min()
        
        date2=df['Date'].max()
        
        col=[col for col in df.columns]
        
        state=compare_header_in_files(col,col_tmp)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
            
        else : 
            
            n+=1
            
            dataframe[n]=df

            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('')
    
    print('### 2. Pre-process dataframes and Storing to database.')

    print('')

    for key, df in dataframe.items():
        
        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')


def tiki_seller_center(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0
    
    print('### 1. Process files ')
    
    print('')
    
    for i in p:
        
        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i,dtype=str)
        
        col=[col for col in df.columns]

        date=i.split('.')[-2]
        
        date=date[:4]+'-'+date[4:6]+'-'+date[6:]
        
        df['Date']=[date]*df.shape[0]
        
        date1=df['Date'].min()
        
        date2=df['Date'].max()
        
        state=compare_header_in_files(col,col_tmp)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
            
        else : 
            
            n+=1
            
            dataframe[n]=df

            warning(date1,date2)
            
            print('')
            
            col_tmp=col
        
    conclude(n)
    
    print('')
    
    print('### 2. Pre-process dataframes and Storing to database.')

    print('')
    
    for key, df in dataframe.items():
        
        unidecode_header(df)
        
        print('')
        
        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')



### DEV_MEDIA_SPENDING

In [15]:
import pandas as pd

import glob

from collections import Counter

from dev_functions import*

import warnings
warnings.filterwarnings('ignore')

db='media_spending_raw'

### LAZ
def laz_aff(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i)
        
        col=[col for col in df.columns]
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        update_to_mysql(db,df,label,Date)
        
        print('')


def laz_search(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i)

        if df[df.columns[0]][4]=='Date':

            df=pd.read_excel(i, header=5, dtype=str)
        
        col=[col for col in df.columns]
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')


def laz_cpas_fb(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')
    
    col_tmp=[]

    dataframe={}

    n=0
    
    print('### 1. Process files ')
    
    print('')
    
    for i in p:
        
        print('file : ', i)
        
        print('')
        
        df=pd.read_csv(i, header=0, dtype=str)
        
        col=[col for col in df.columns]
        
        df = df.rename(columns={"Reporting starts": "Date",})

        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]

        state=compare_header_in_files(col,col_tmp)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
            
        else : 
            
            n+=1
            
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col

    conclude(5)
    
    print('')
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
        
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')
        
        update_to_mysql(db,df,label,Date)
        
        print('')


def laz_cpas_msb_sku(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')
    
    col_tmp=[]

    dataframe={}

    n=0
    
    print('### 1. Process files ')
    
    print('')
    
    for i in p:
        
        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i, header=0, sheet_name='SKU Performance', dtype=str)
        
        col=[col for col in df.columns]
    
        df['Date']=[x[:4]+'-'+x[4:6]+'-'+x[6:] for x in df['Date'].tolist()]
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]

        state=compare_header_in_files(col,col_tmp)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
            
        else : 
            
            n+=1
            
            dataframe[n]=df

            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('')
    
    print('### 2. Pre-process dataframes and Storing to database.')

    print('')

    for key, df in dataframe.items():
        
        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        update_to_mysql(db,df,label,Date)
        
        print('')


def laz_cpas_msb_overview(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0
    
    print('### 1. Process files ')
    
    print('')
    
    for i in p:
        
        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i, header=0, sheet_name='Overview', dtype=str)

        col=[col for col in df.columns]

        df['Date']=[x[:4]+'-'+x[4:6]+'-'+x[6:] for x in df['Date'].tolist()]
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col,col_tmp)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
            
        else : 
            
            n+=1
            
            dataframe[n]=df

            warning(date1,date2)
            
            print('')
            
            col_tmp=col
        
    conclude(n)
    
    print('')
    
    print('### 2. Pre-process dataframes and Storing to database.')

    print('')
    
    for key, df in dataframe.items():
        
        unidecode_header(df)
        
        print('')
        
        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        update_to_mysql(db,df,label,Date)
        
        print('')


###     SHP 
def shp_aff(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_csv(i,header=0)
        
        col=[col for col in df.columns]

        df = df.rename(columns={'Purchase Time': "Date",})
        
        tmp=[]
        
        for date in df['Date']:
            
            date=date[:10]
            
            date=covert_date_str(date)
            
            tmp.append(date)

        df['Date']=tmp
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        update_to_mysql(db,df,label,Date)
        
        print('')


def shp_search(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i)
        
        col=[col for col in df.columns]
        
        date=i.split('B-')[-1].split('.xl')[0].replace('.','-')

        df['Date']=[date]*df.shape[0]
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        update_to_mysql(db,df,label,Date)
        
        print('')


def shp_cpas_fb(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_csv(i)
        
        col=[col for col in df.columns]
        
        df = df.rename(columns={'Reporting starts': "Date",})
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')
        
        
def shp_cpas_msb_sku(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i, sheet_name='By Day', header=0)

        col=[col for col in df.columns]

        df['Date']=[x[6:]+'-'+x[3:5]+'-'+x[0:2] for x in df['Date'].tolist()]
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')
      

def shp_cpas_msb_camp(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i, sheet_name='Details Sheet', header=0,skiprows=[1])
        
        col=[col for col in df.columns]
        
        df = df.rename(columns={'Date(DD/MM/YYYY)': "Date",})
        
        tmp=[]
        
        for date in df['Date']:
            
            date=date[:10]
            
            date=covert_date_str(date)
            
            tmp.append(date)

        df['Date']=tmp
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')


### TIKI 

def tiki_aff(db,p,label):
    
    from openpyxl import load_workbook
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
    
        df=[]
        
        a=[]
        
        for i in p:
        
            wb = load_workbook(i, read_only=True, keep_links=False)
            
            for x in wb.sheetnames:
                
                y=pd.read_excel(i, sheet_name=x, dtype=str, skiprows=0)

                a.append(y)

        df=pd.concat(a)
        
        col=[col for col in df.columns]
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')


def tiki_search(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_excel(i)
        
        col=[col for col in df.columns]
        
        df = df.rename(columns={'Thời gian': "Date",})
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        DDate=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')
        

def tiki_cpas_fb(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_csv(i)
        
        col=[col for col in df.columns]
        
        df = df.rename(columns={'Reporting starts': "Date",})
        
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')
     
  
def tiki_cpas_msb_camp(db,p,label):
    
    print('Pipeline : ' , label)
    
    print('')

    col_tmp=[]

    dataframe={}

    n=0

    print('### 1. Process files ')
    
    print('')
    
    for i in p:

        print('file : ', i)
        
        print('')
        
        df=pd.read_csv(i)
        
        col=[col for col in df.columns]
       
        df['Date Time']=[str(x).split()[0] for x in df['Date Time']]
        
        df = df.rename(columns={'Date Time': "Date",})
 
        date1=df['Date'].min()
        
        date1=date1[:10]
        
        date2=df['Date'].max()
        
        date2=date2[:10]
        
        state=compare_header_in_files(col_tmp,col)
        
        print('')
        
        if state==True:
            
            dataframe[n]=pd.concat([dataframe[n],df])
          
        else : 
           
            n+=1
           
            dataframe[n]=df
            
            warning(date1,date2)
            
            print('')
            
            col_tmp=col
    
    conclude(n)
    
    print('### 2. Pre-process dataframes and Storing to database.')
    
    print('')
    
    for key, df in dataframe.items():

        prepare_header_to_mysql(df)
        
        print('')
        
        df=df.sort_values('DATE').drop_duplicates()
        
        Date=store_data_to_local(db,df,label)
        
        Date=Date[:8]
        
        print('')

        #update_to_mysql(db,df,label,Date)
        
        print('')
      


### TEST MEDIA_SPENDING : lazada

Lazada

In [16]:
p=glob.glob('data/platform_lazada/affiliate/raw/*')

p.sort()

laz_aff('media_spending_raw',p,'laz_aff')

Pipeline :  laz_aff

### 1. Process files 

file :  data/platform_lazada/affiliate/raw/ItemPerformanceData_20220101_20220731.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-07-31

**************************************************

file :  data/platform_lazada/affiliate/raw/ItemPerformanceData_20220801_20220831.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/affiliate/raw/ItemPerformanceData_20220901_20221016.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/affiliate/raw/ItemPerformanceData_20221017_20221210.xlsx

***** { Function } : compare_header_in_files() 

---> True 

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   1 dataframes.

__________________________________________________

In [17]:
p=glob.glob('data/platform_lazada/search/raw/*')

p.sort()

laz_search('media_spending_raw',p,'laz_search')

Pipeline :  laz_search

### 1. Process files 

file :  data/platform_lazada/search/raw/2022-01-01--2022-01-1009_56.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-01-10

**************************************************

file :  data/platform_lazada/search/raw/2022-01-11--2022-01-2009_57.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/search/raw/2022-01-21--2022-01-3109_57.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/search/raw/2022-02-01--2022-02-1010_00.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/search/raw/2022-02-11--2022-02-2010_00.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/search/raw/2022-02-21--2022-02-2810_00.xlsx

***** { Func

In [22]:
p=glob.glob('data/platform_lazada/cpas/fb/raw/*.csv')

p.sort()

laz_cpas_fb('media_spending_raw',p,'laz_cpas_fb')

Pipeline :  laz_cpas_fb

### 1. Process files 

file :  data/platform_lazada/cpas/fb/raw/PMAX-Tefal-CPAS-LZD-Campaigns-1-Jan-202216-Oct-2022.csv

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-10-16

**************************************************

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   5 dataframes.

________________________________________________________________________________________________________________________________________________________________________


### 2. Pre-process dataframes and Storing to database.

***** { Function } : prepare_header_to_mysql() 


***** { Function } : store_data_to_local() 

---> local :  data/output_key_metrics/laz_cpas_fb_20220101.xlsx

dataframe-head : 
              DATE REPORTING_ENDS  \
97970  2022-01-01     2022-01-01   
9

In [23]:
p=glob.glob('data/platform_lazada/cpas/msb/raw/*')

p.sort()

laz_cpas_msb_sku('media_spending_raw',p,'laz_cpas_msb_sku')

Pipeline :  laz_cpas_msb_sku

### 1. Process files 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2201.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-01-31

**************************************************

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2202.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2203.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2204.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2205.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/pl

In [24]:
p=glob.glob('data/platform_lazada/cpas/msb/raw/*')

p.sort()

laz_cpas_msb_overview('media_spending_raw',p,'laz_cpas_msb_overview')

Pipeline :  laz_cpas_msb_overview

### 1. Process files 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2201.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-01-31

**************************************************

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2202.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2203.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2204.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/cpas/msb/raw/LAZADA-Campaign_Offer_Links_Report2205.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  da

### TEST MEDIA_SPENDING : Shopee

Shopee

In [25]:
p=glob.glob('data/platform_shopee/affiliate/raw/*.csv')

p.sort()

shp_aff(db,p,'shp_aff')



Pipeline :  shp_aff

### 1. Process files 

file :  data/platform_shopee/affiliate/raw/Shopee_Affiliate_20220101_20220531.csv

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-14 -  2022-05-22

**************************************************

file :  data/platform_shopee/affiliate/raw/Shopee_Affiliate_20220601_20220731.csv

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-06-01 -  2022-07-31

**************************************************

file :  data/platform_shopee/affiliate/raw/Shopee_Affiliate_20220801_20221016.csv

***** { Function } : compare_header_in_files() 

---> True 

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   2 dataframes.

__________________

In [27]:
p=glob.glob('data/platform_shopee/search/raw/*')

p.sort()

shp_search(db,p,'shp_search')

Pipeline :  shp_search

### 1. Process files 

file :  data/platform_shopee/search/raw/On-platform_Ads_Report---Product_Performance---Groupe-SEB-2022.01.01.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-01-01

**************************************************

file :  data/platform_shopee/search/raw/On-platform_Ads_Report---Product_Performance---Groupe-SEB-2022.01.02.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/search/raw/On-platform_Ads_Report---Product_Performance---Groupe-SEB-2022.01.03.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/search/raw/On-platform_Ads_Report---Product_Performance---Groupe-SEB-2022.01.04.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/search/raw/On-platform_Ads_R

In [31]:
p=glob.glob('data/platform_shopee/cpas/fb/raw/*.csv')

p.sort()

shp_cpas_fb(db,p,'shp_cpas_fb')

Pipeline :  shp_cpas_fb

### 1. Process files 

file :  data/platform_shopee/cpas/fb/raw/PMAX-Tefal-Shopee-Campaigns-1-Jan-202216-Oct-2022.csv

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-10-16

**************************************************

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   1 dataframes.

________________________________________________________________________________________________________________________________________________________________________

### 2. Pre-process dataframes and Storing to database.

***** { Function } : prepare_header_to_mysql() 


***** { Function } : store_data_to_local() 

---> local :  data/output_key_metrics/shp_cpas_fb_20220101.xlsx

dataframe-head : 
              DATE REPORTING_ENDS  \
91612  2022-01-01     2022-01-01   
9139

In [32]:
p=glob.glob('data/platform_shopee/cpas/msb/raw/sku/*')

p.sort()

shp_cpas_msb_sku(db,p,'shp_cpas_msb_sku')

Pipeline :  shp_cpas_msb_sku

### 1. Process files 

file :  data/platform_shopee/cpas/msb/raw/sku/Off-platform_Traffic_Report---Product_Performance---Groupe-SEB-2022.01.01-2022.10.16.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-10-16

**************************************************

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   1 dataframes.

________________________________________________________________________________________________________________________________________________________________________

### 2. Pre-process dataframes and Storing to database.

***** { Function } : prepare_header_to_mysql() 


***** { Function } : store_data_to_local() 

---> local :  data/output_key_metrics/shp_cpas_msb_sku_20220101.xlsx

dataframe-head : 
             DATE REGION  

In [37]:
p=glob.glob('data/platform_shopee/cpas/msb/raw/campaign/*')

p.sort()

shp_cpas_msb_camp(db,p,'shp_cpas_msb_camp')

Pipeline :  shp_cpas_msb_camp

### 1. Process files 

file :  data/platform_shopee/cpas/msb/raw/campaign/Off-platform_Traffic_Report---Campaign_Performance---Groupe-SEB-2022.01.01-2022.10.16.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-10-16

**************************************************

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   1 dataframes.

________________________________________________________________________________________________________________________________________________________________________

### 2. Pre-process dataframes and Storing to database.

***** { Function } : prepare_header_to_mysql() 


***** { Function } : store_data_to_local() 

---> local :  data/output_key_metrics/shp_cpas_msb_camp_20220101.xlsx

dataframe-head : 
              DATE

### TEST MEDIA_SPENDING : Tiki

In [39]:
p=glob.glob('data/platform_tiki/affiliate/raw/*')

p.sort()

tiki_aff(db,p,'tiki_aff')


Pipeline :  tiki_aff

### 1. Process files 

file :  data/platform_tiki/affiliate/raw/AFF Tefal Jan - Sep.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-09-30

**************************************************

file :  data/platform_tiki/affiliate/raw/program-2f082d2c-b9e3-4acb-b7ac-5e7b0827b365-2022-07-29-2022-10-26.xlsx

***** { Function } : compare_header_in_files() 

---> True 

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   1 dataframes.

________________________________________________________________________________________________________________________________________________________________________

### 2. Pre-process dataframes and Storing to database.

***** { Function } : prepare_header_to_mysql() 


***** { Function } : store_data_to_local() 

---> local :  dat

In [11]:
p=glob.glob('data/platform_tiki/search/raw/product_20220101_20221031.xlsx')

p.sort()

tiki_search(db,p,'tiki_search')

Pipeline :  tiki_search

### 1. Process files 

file :  data/platform_tiki/search/raw/product_20220101_20221031.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-10-31

**************************************************

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   1 dataframes.

________________________________________________________________________________________________________________________________________________________________________

### 2. Pre-process dataframes and Storing to database.

***** { Function } : prepare_header_to_mysql() 

***** { Function } : unidecode_header() 


***** { Function } : store_data_to_local() 

---> local :  data/output_key_metrics/tiki_search_20220101.xlsx

dataframe-head : 
            DATE     LOAI_QUANG_CAO           TEN_CHIEN_DICH 

In [4]:
p=glob.glob('data/platform_tiki/cpas/fb/raw/*.csv')

p.sort()

tiki_cpas_fb(db,p,'tiki_cpas_fb')

Pipeline :  tiki_cpas_fb

### 1. Process files 

file :  data/platform_tiki/cpas/fb/raw/PMAX-Tefal-Tiki-Campaigns-1-Jan-202216-Oct-2022.csv

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-10-16

**************************************************

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   1 dataframes.

________________________________________________________________________________________________________________________________________________________________________

### 2. Pre-process dataframes and Storing to database.

***** { Function } : prepare_header_to_mysql() 

***** { Function } : unidecode_header() 


***** { Function } : store_data_to_local() 

---> local :  data/output_key_metrics/tiki_cpas_fb_20220101.xlsx

dataframe-head : 
              DATE REPORTING_ENDS  \


In [10]:
p=glob.glob('data/platform_tiki/cpas/msb/raw/*.csv')

p.sort()

tiki_cpas_msb_camp(db,p,'tiki_cpas_msb_camp')

Pipeline :  tiki_cpas_msb_camp

### 1. Process files 

file :  data/platform_tiki/cpas/msb/raw/report-campaign-1666842012026.csv

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-07-29 -  2022-10-26

**************************************************

file :  data/platform_tiki/cpas/msb/raw/report-product-1666841999408.csv

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-07-29 -  2022-10-26

**************************************************

 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _ 

===> total :   2 dataframes.

________________________________________________________________________________________________________________________________________________________________________

### TEST KEY_METRICS 

In [10]:
p=glob.glob('data/platform_lazada/business_advisor/raw/*/*')
    
p.sort()
    
laz_biz(db,p,'laz_biz')


Pipeline :  laz_biz

### 1. Process files 

file :  data/platform_lazada/business_advisor/raw/20220201-20220228/01022022.xls

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-02-01 -  2022-02-01

**************************************************

file :  data/platform_lazada/business_advisor/raw/20220201-20220228/02022022.xls

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/business_advisor/raw/20220201-20220228/03022022.xls

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/business_advisor/raw/20220201-20220228/04022022.xls

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/business_advisor/raw/20220201-20220228/05022022.xls

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_lazada/business_advisor/raw

In [11]:
p=glob.glob('data/platform_shopee/business_advisor/product/raw/*')
    
p.sort()
    
shp_biz(db,p,'shp_biz')

Pipeline :  shp_biz

### 1. Process files 

file :  data/platform_shopee/business_advisor/product/raw/export_report.parentskudetail.20220101_20220101.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  20220101 -  20220101

**************************************************

file :  data/platform_shopee/business_advisor/product/raw/export_report.parentskudetail.20220102_20220102.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/business_advisor/product/raw/export_report.parentskudetail.20220103_20220103.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/business_advisor/product/raw/export_report.parentskudetail.20220104_20220104.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/business_advisor/product/raw/export_report.pare

In [12]:
p=glob.glob('data/platform_shopee/business_advisor/overall/raw/*')
    
p.sort()
    
shp_overall(db,p,'shp_overall')

Pipeline :  shp_overall

### 1. Process files 

file :  data/platform_shopee/business_advisor/overall/raw/tefal_vn_official.shopee-shop-stats.20210101-20210131.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2021-01-01 -  2021-01-31

**************************************************

file :  data/platform_shopee/business_advisor/overall/raw/tefal_vn_official.shopee-shop-stats.20210201-20210228.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/business_advisor/overall/raw/tefal_vn_official.shopee-shop-stats.20210301-20210331.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/business_advisor/overall/raw/tefal_vn_official.shopee-shop-stats.20210401-20210430.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_shopee/business_adviso

In [13]:
p=glob.glob('data/platform_tiki/business_advisor/seller_center/raw/*')
    
p.sort()
    
tiki_seller_center(db,p,'tiki_seller_center')

Pipeline :  tiki_seller_center

### 1. Process files 

file :  data/platform_tiki/business_advisor/seller_center/raw/Tefal.Official.Store.-.Chính.hãng.product-stats.20220101.20220101.xlsx

***** { Function } : compare_header_in_files() 

---> False 

**************************************************

 ===> Warning : file changes header-columns name !

 Date :  2022-01-01 -  2022-01-01

**************************************************

file :  data/platform_tiki/business_advisor/seller_center/raw/Tefal.Official.Store.-.Chính.hãng.product-stats.20220102.20220102.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_tiki/business_advisor/seller_center/raw/Tefal.Official.Store.-.Chính.hãng.product-stats.20220103.20220103.xlsx

***** { Function } : compare_header_in_files() 

---> True 

file :  data/platform_tiki/business_advisor/seller_center/raw/Tefal.Official.Store.-.Chính.hãng.product-stats.20220104.20220104.xlsx

***** { Function } : compare_heade